In [ ]:
import pysam

In [ ]:
# ~~~~~ filtering PCR duplicates from a bam file ~~~~~ #

input_bam = "input.bam"
output_bam = "filtered_no_duplicates.bam"
sorted_output_bam = "demuxed_aligned_no_dup_sorted.bam"

with pysam.AlignmentFile(input_bam, "rb") as infile: # Open input BAM file in read mode
    with pysam.AlignmentFile(output_bam, "wb", header=infile.header) as outfile: # Create output BAM with same header
        for read in infile:
            if not read.is_duplicate: # Only write reads that are NOT PCR duplicates
                outfile.write(read)

pysam.sort("-@", "4", "-o", sorted_output_bam, output_bam)
pysam.index(sorted_output_bam) # Index the output BAM 
os.remove(output_bam)


In [ ]:
# ~~~~~ etxracting a sepecific tag value from a bam file ~~~~~ #

with pysam.AlignmentFile(bam_file, "rb") as bam:
        for read in bam:
            read_name = read.query_name
            duplicate_tag = read.get_tag("DT") if read.has_tag("DT") else "No"


In [ ]:
# ~~~~~ merging bam files ~~~~~~ #

def merge_bam_files(output_bam, input_folder):
    """Merges temporary BAM files into a single output BAM."""
    input_files = [f for f in os.listdir(input_folder) if f.endswith(".bam")]
    merged_bam = output_bam
    
    with pysam.AlignmentFile(input_files[0], "rb") as template:
        with pysam.AlignmentFile(merged_bam, "wb", template=template) as merged_out:
            for temp_file in input_files:
                with pysam.AlignmentFile(temp_file, "rb") as temp_bam:
                    for read in temp_bam:
                        merged_out.write(read)
                os.remove(temp_file)  # Clean up temporary BAM files


In [ ]:
# ~~~~~ read lengths from the bam file ~~~~~ #

sorted_bam = "input.bam"
cDNA_lengths = []

with pysam.AlignmentFile(sorted_output_bam, 'rb') as no_dup_bam:
    for read in no_dup_bam:
        if (not read.is_secondary) and (not read.is_supplementary):
            cDNA_lengths.append(read.query_length)
